## 2. mongodb 인덱스 (INDEX)
- 검색을 더 빠르게 수행하고자 만든 추가적인 data structure
  - index 데이터 구조가 없다면, 컬렉션에 있는 데이터를 하나씩 조회하는 방식으로 검색이 됨

### 2.1 기본 인덱스 _id
- 모든 mongodb 컬렉션은 기본적으로 _id 필드를 기반으로 기본 인덱스가 생성됨

#### mongodb 접속 기본 코드

In [2]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print (doc)
actor = actor_collection

{'_id': ObjectId('6019112ffd9a8148c2183d1a'), '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트', '배우이름': '황정민', '흥행지수': 78066, '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'], '랭킹': 1}
{'_id': ObjectId('6019112ffd9a8148c2183d1b'), '직업': '배우', '성별': '남', '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n', '소속사': '아티스트컴퍼니', '배우이름': '이정재', '흥행지수': 52049, '출연영화': ['다만 악에서 구하소서', '신세계', '신과 함께-죄와 벌', '신과 함께-인과 연', '사바하', '도둑들'], '랭킹': 2}
{'_id': ObjectId('6019112ffd9a8148c2183d1c'), '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/hongsick/\nhttps://twitter.com/seeksik\n', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영', '배우이름': '유아인', '흥행지수': 42530, '출연영화': ['#살아있다', '소리도 없이', '버닝', '사도'], '랭킹': 3, '다른이름': '엄홍식'}


### 2.2 Single(단일) 필드 인덱스

#### index 생성: create_index()

In [26]:
actor.create_index('배우이름')

'배우이름_1'

#### index 정보 확인: index_information()

In [27]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]}}

- key: ('필드명', direction)
  - direction
    - pymongo.ASCENDING = 1
    - pymongo.DESCENDING = -1
    - pymongo.TEXT = 'text'

#### 전체 인덱스 삭제

In [4]:
actor.drop_indexes()

In [25]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

#### 특정 인덱스 삭제

In [12]:
actor.drop_index([('배우이름', 1)])

In [29]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

#### 여러개의 single index 생성 가능

In [14]:
actor.create_index('배우이름')

'배우이름_1'

In [15]:
actor.create_index('랭킹')

'랭킹_1'

In [16]:
actor.create_index('흥행지수')

'흥행지수_1'

In [19]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)]},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)]},
 '직업_-1': {'v': 2, 'key': [('직업', -1)]}}

In [18]:
actor.create_index([('직업', pymongo.DESCENDING)])

'직업_-1'

#### 부분 문자열 검색을 위해, text index 생성

In [41]:
actor.drop_indexes()

In [42]:
actor.create_index([('출연영화', 'text')])

'출연영화_text'

In [43]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '출연영화_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

#### 부분 문자열 검색: $text

In [50]:
docs = actor.find({'$text': { '$search': '다만' }})
for doc in docs:
    print(doc)

{'_id': ObjectId('6019112ffd9a8148c2183d1f'), '직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과', '배우이름': '박정민', '흥행지수': 26048, '출연영화': ['다만 악에서 구하소서', '시동', '사냥의 시간', '들개', '동주', '사바하'], '랭킹': 6}
{'_id': ObjectId('6019112ffd9a8148c2183d1b'), '직업': '배우', '성별': '남', '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n', '소속사': '아티스트컴퍼니', '배우이름': '이정재', '흥행지수': 52049, '출연영화': ['다만 악에서 구하소서', '신세계', '신과 함께-죄와 벌', '신과 함께-인과 연', '사바하', '도둑들'], '랭킹': 2}
{'_id': ObjectId('6019112ffd9a8148c2183d1a'), '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과 졸업', '특기': '농구, 악기연주', '소속사': '예당엔터테인먼트', '배우이름': '황정민', '흥행지수': 78066, '출연영화': ['다만 악에서 구하소서', '신세계', '부당거래', '국제시장', '아수라'], '랭킹': 1}


In [41]:
docs = actor.find({'$text': {'$search': '가수'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('5d4541ccc92b652d52161387'), '배우이름': '임윤아', '흥행지수': 2176, '출연영화': ['엑시트', '공조'], '랭킹': '117', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}
{'_id': ObjectId('5d4541ccc92b652d52161378'), '배우이름': '진영', '흥행지수': 2469, '출연영화': ['내안의 그놈', '수상한 그녀'], '랭킹': '102', '직업': '가수', '생년월일': '1991-11-18', '성별': '남', '홈페이지': '\nhttps://twitter.com/_jinyoung911118\nhttps://instagram.com/jinyoung0423/\n', '신장/체중': '178cm, 59kg', '다른이름': '정진영; 비원에이포; B1A4'}
{'_id': ObjectId('5d4541ccc92b652d52161377'), '배우이름': '도경수', '흥행지수': 2475, '출연영화': ['스윙키즈', '언더독', '신과 함께-죄와 벌', '형', '순정', '7호실'], '랭킹': '101', '직업': '가수', '성별': '남', '다른이름': '엑소;EXO; 디오'}
{'_id': ObjectId('5d4541ccc92b652d52161369'), '배우이름': '김설현', '흥행지수': 2944, '출연영화': ['안시성'], '랭킹': '87', '직업': '가수', '생년월일': '1995-01-03', '성별': '여', '홈페이지': '\nhttps://instagram.com/sh_9513\n', '다른이름': '김설현;AOA;에이오에이'}


### 2.3 Compound(복합) 필드 인덱스
- 최대 31개의 필드로 만들 수 있음

In [8]:
actor.drop_indexes()

In [82]:
actor.create_index([('출연영화', pymongo.TEXT), ('직업', pymongo.TEXT), ('흥행지수', pymongo.DESCENDING)])

'출연영화_text_직업_text_흥행지수_-1'

In [6]:
docs = actor.find({'$text': {'$search': '신과'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('6019112ffd9a8148c2183f4f'), '직업': '배우', '생년월일': '1983-07-29', '성별': '남', '신장/체중': '175cm, 60kg', '학교': '한국예술종합학교 연극원', '취미': '음악듣기, 축구', '특기': '기계체조', '배우이름': '김동욱', '흥행지수': 13, '출연영화': ['신과 함께-죄와 벌', '신과 함께-인과 연'], '랭킹': 566}
{'_id': ObjectId('6019112ffd9a8148c2183f3f'), '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': '\nhttps://twitter.com/smell2001\n', '특기': '애교 부리기, 피아노 치기', '배우이름': '김향기', '흥행지수': 15, '출연영화': ['증인', '신과 함께-죄와 벌', '신과 함께-인과 연'], '랭킹': 550}
{'_id': ObjectId('6019112ffd9a8148c2183f30'), '원어명': '주지훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노', '배우이름': '주지훈', '흥행지수': 16, '출연영화': ['아수라', '신과 함께-죄와 벌', '신과 함께-인과 연'], '랭킹': 535, '다른이름': '주영훈'}
{'_id': ObjectId('6019112ffd9a8148c2183df6'), '직업': '배우', '생년월일': '1984-03-13', '성별': '남', '홈페이지': '\nhttps://twitter.com/4eyedjack\n', '신장/체중': '181cm, 70kg', '학교': '한신대학

### 2.4 다양한 문자열 검색

In [9]:
# 정규표현식 ($text operator 는 $search operator 와 함께 사용됨)
result = actor.find({'출연영화' : {'$regex' : '함께'}})
for record in result:
    print(record)

{'_id': ObjectId('6019112ffd9a8148c2183d1b'), '직업': '배우', '성별': '남', '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n', '소속사': '아티스트컴퍼니', '배우이름': '이정재', '흥행지수': 52049, '출연영화': ['다만 악에서 구하소서', '신세계', '신과 함께-죄와 벌', '신과 함께-인과 연', '사바하', '도둑들'], '랭킹': 2}
{'_id': ObjectId('6019112ffd9a8148c2183d21'), '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '배우이름': '하정우', '흥행지수': 22609, '출연영화': ['클로젯', '백두산', '범죄와의 전쟁 : 나쁜 놈들 전성시대', '멋진 하루', '아가씨', '신과 함께-죄와 벌'], '랭킹': 8, '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('6019112ffd9a8148c2183d42'), '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트', '배우이름': '임원희', '흥행지수': 5515, '출연영화': ['천문: 하늘에 묻는다', '신과 함께-죄와 벌', '도굴', '신과 함께-인과 연'], '랭킹': 41}
{'_id': ObjectId('6019112ffd9a8148c2183d58'), '직업': '배우', '생년월일': '1952-01-05', '성별': '남', '배우이름': '장광', '흥행지수': 3632, '출연영화'

In [10]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
text_collection = actor_db.text_collection

In [11]:
text_collection.insert_many(
    [
        { "name": "Java Hut", "description": "Coffee and cakes", "ranking": 1 },
        { "name": "Burger Buns", "description": "Java hamburgers", "ranking": 2 },
        { "name": "Coffee Shop", "description": "Just coffee", "ranking": 3 },
        { "name": "Clothes Clothes Clothes", "description": "Discount clothing", "ranking": 4 },
        { "name": "Java Shopping", "description": "Indonesian goods", "ranking": 5 }
    ]
)

In [13]:
docs = text_collection.find({'name': {'$regex': 'Co.*'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [18]:
docs = text_collection.find({'name': {'$regex': 'Sh.*'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('601be600a27b1dc5d5becb2f'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [19]:
text_collection.create_index([('name', pymongo.TEXT)])

'name_text'

In [20]:
docs = text_collection.find({'$text': {'$search': 'coffee'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [21]:
docs = text_collection.find({'$text': {'$search': 'java coffee shop'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('601be600a27b1dc5d5becb2f'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}
{'_id': ObjectId('601be600a27b1dc5d5becb2b'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}


In [26]:
docs = text_collection.find({'$text': {'$search': '\"coffee shop\"'}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [27]:
# 문자열 대소문자 구분 '$caseSensitive': True
docs = text_collection.find({'$text': {'$search': 'Coffee', '$caseSensitive': True}})
for doc in docs:
    print (doc)

{'_id': ObjectId('601be600a27b1dc5d5becb2d'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
배우중에 중앙대학교를 나온 배우를 흥행지수 순으로 최대 10명 출력하라
</div>

In [106]:
result = actor.find({'학교' : {'$regex' : '중앙대학교'}}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for record in result:
    print(record)

{'_id': ObjectId('5d4541ccc92b652d52161313'), '배우이름': '주지훈', '흥행지수': 29515, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'], '랭킹': '1', '원어명': '주지훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노', '다른이름': '주영훈'}
{'_id': ObjectId('5d4541ccc92b652d52161357'), '배우이름': '정우성', '흥행지수': 3732, '출연영화': ['인랑', '증인', '호우시절', '더 킹', '비트', '아수라'], '랭킹': '69', '직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws\n', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름'}


In [41]:
results = actor.find({'학교': {'$regex': '경희'}}, {'배우이름': 1, '학교': 1, '_id': 0}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for result in results:
    print(result)

{'학교': '경희대언론정보대학원 문화콘텐츠학', '배우이름': '정준호'}
{'학교': '총신대학교 종교음악과,경희대학교대학원', '배우이름': '김선경'}
{'학교': '경희대학교 전자공학과', '배우이름': '강신일'}
{'학교': '경희대학교에서 철학과/연극영화과 전공', '배우이름': '박수연'}
{'학교': '경희대학교 간호학 (중퇴)', '배우이름': '김해숙'}
{'학교': '경희대학교 언론정보대학원', '배우이름': '김지미'}
{'학교': '경희대학교 연극영화 - 경희대아트퓨전디자인대학원 퍼포밍아트학', '배우이름': '공유'}
{'학교': '경희대학교 연극영화', '배우이름': '김옥빈'}
{'학교': '경희대학교 연극영화학 재학', '배우이름': '이홍기'}
{'학교': '경희대학교 연극영화과', '배우이름': '전재형'}
